In [10]:
%load_ext autoreload
%autoreload 2

In [11]:

import math
import time
from tqdm import tqdm



import torch
import numpy as np
import torch.nn as nn
from torch import Tensor

from torch.utils.data import DataLoader
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)

from src.prepare_data import download_data, build_train_vocab, get_train_test_val, check_tokens,tokens_to_sentence , generate_batch , visualize_iter_data , get_embed
from src.LSTM import RNNdecoder, RNNencoder,Seq2SeqRNN
from src.train import create_mask,generate_square_subsequent_mask, train_epoch, bleu_calculate , evaluate
from src.transformer import Seq2SeqTransformer,PositionalEncoding,TokenEmbedding


In [45]:
# from transformers import AutoModel, AutoTokenizer

In [12]:
from src.LSTM_seminars import seq2seq
from src.train_seminars import trainer

In [14]:
train_filepaths , val_filepaths , test_filepaths = download_data()
de_vocab, en_vocab, de_tokenizer, en_tokenizer = build_train_vocab(train_filepaths)
print( 'De vocab En vocab: ',len(de_vocab), len(en_vocab))
train_data , val_data , test_data = get_train_test_val(train_filepaths, test_filepaths, val_filepaths , de_vocab , en_vocab ,de_tokenizer,en_tokenizer )
print('Train Test Val: ',len(train_data),len(test_data) , len(val_data))

De vocab En vocab:  19215 10838
Train Test Val:  29000 1014 1000


In [15]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']
print(PAD_IDX , BOS_IDX , EOS_IDX)

1 2 3


In [16]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn= lambda x : generate_batch(x , BOS_IDX=BOS_IDX,PAD_IDX=PAD_IDX,EOS_IDX=EOS_IDX))
valid_iter = DataLoader(val_data, batch_size=1,
                        shuffle=True, collate_fn= lambda x : generate_batch(x , BOS_IDX=BOS_IDX,PAD_IDX=PAD_IDX,EOS_IDX=EOS_IDX))
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn= lambda x : generate_batch(x , BOS_IDX=BOS_IDX,PAD_IDX=PAD_IDX,EOS_IDX=EOS_IDX))

In [17]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

In [18]:
SRC_VOCAB_SIZE = len(de_vocab)
TGT_VOCAB_SIZE = len(en_vocab)

In [19]:
model = seq2seq(SRC_VOCAB_SIZE)
model.to(DEVICE)

seq2seq(
  (encoder): Encoder(
    (embedding): Embedding(19215, 30)
    (lstm): LSTM(30, 30, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(19215, 30)
    (attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=30, out_features=30, bias=True)
    )
    (lstm): LSTM(30, 30, batch_first=True)
    (linear): Linear(in_features=30, out_features=19215, bias=True)
  )
)

In [20]:
loss_function = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam


In [37]:
train_data[0][0].tolist()

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [42]:
de_train_dataset = []
en_train_dataset = []
for j in train_data :
    de_train_dataset.append(j[0].tolist())
    en_train_dataset.append(j[1].tolist())
de_train_dataset = np.array(de_train_dataset, dtype=object)
en_train_dataset = np.array(en_train_dataset, dtype=object)

In [47]:
from torch.nn.utils.rnn import pad_sequence

In [50]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_data, 
shuffle=True, pin_memory=True, batch_size=BATCH_SIZE,collate_fn= lambda x : generate_batch(x , BOS_IDX=BOS_IDX,PAD_IDX=PAD_IDX,EOS_IDX=EOS_IDX))

In [54]:
trainer(count_of_epoch=15, 
        batch_size=64, 
        dataloader=train_dataloader,
        dataset = tran_data ,
        model=model, 
        loss_function=loss_function,
        optimizer = optimizer,
        lr=0.001,
        callback=None)

epoch:   0%|          | 0/15 [00:00<?, ?it/s, train epoch loss=nan]


NameError: name 'dataset' is not defined

(tensor([24, 25, 26, 27, 11, 22, 28, 29, 30, 16]),
 tensor([26, 27, 28, 29, 30, 22, 31, 32, 14]))